In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# 1. 定义 CNN 模型
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(128 * 28 * 28, 1024)
        self.fc2 = nn.Linear(1024, 3)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(2, 2)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        x = x.view(-1, 128 * 28 * 28)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return self.softmax(x)

model = SimpleCNN()

In [4]:
# 2. 数据处理
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset_path = 'D:/tester01/训练'  
val_dataset_path = 'D:/tester01/验证' 
test_dataset_path = 'D:/tester01/测试'

train_dataset = datasets.ImageFolder(train_dataset_path, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

val_dataset = datasets.ImageFolder(val_dataset_path, transform=transform)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

test_dataset = datasets.ImageFolder(test_dataset_path, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [5]:
# 3. 模型训练
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

for epoch in range(5):
    model.train()
    for data, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
    
    model.eval()
    total = 0
    correct = 0
    with torch.no_grad():
        for data, targets in val_loader:
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

    print(f'Epoch {epoch+1}, Accuracy: {100 * correct / total}%')

Epoch 1, Accuracy: 87.23404255319149%
Epoch 2, Accuracy: 94.10801963993454%
Epoch 3, Accuracy: 94.10801963993454%
Epoch 4, Accuracy: 94.76268412438625%
Epoch 5, Accuracy: 95.90834697217676%


In [6]:
model.eval()
total_test = 0
correct_test = 0
with torch.no_grad():
    for data, targets in test_loader:
        outputs = model(data)
        _, predicted = torch.max(outputs.data, 1)
        total_test += targets.size(0)
        correct_test += (predicted == targets).sum().item()

print(f'Test Accuracy: {100 * correct_test / total_test}%')

Test Accuracy: 94.92635024549918%


In [7]:
torch.save(model, '白内障model.pth')